In [1]:
import os
import os.path as osp
import torch
import pandas as pd
import numpy as np
from skimage import io
import warnings
warnings.filterwarnings("ignore")
import imageio
import matplotlib.pyplot as plt


class Postprocess(object):
    def __init__(self,
                 is_3d,
                 type_masks,
                 merge_operation,
                 decision_threshold,
                 path_inference_output,
                 center_coord,
                 path_seg_result,
                 directed=True
                 ):

        file1 = os.path.join(path_inference_output, 'pytorch_geometric_data.pt')
        file2 = os.path.join(path_inference_output, 'all_data_df.csv')
        file3 = os.path.join(path_inference_output, 'raw_output.pt')

        self.dir_result = dir_results = path_seg_result
        self.results = []
        if os.path.exists(dir_results):
            self.results = [os.path.join(dir_results, fname) for fname in sorted(os.listdir(dir_results))
                            if type_masks in fname]

        self.is_3d = is_3d
        self.center_coord = center_coord
        self.merge_operation = merge_operation
        self.decision_threshold = decision_threshold
        self.directed = directed
        self.path_inference_output = path_inference_output
        self.cols = ["child_id", "parent_id", "start_frame"]

        data = self._load_file(file1)
        self.edge_index = data.edge_index

        self.df_preds = self._load_file(file2)
        self.output_pred = self._load_file(file3)
        self.find_connected_edges()

    def _load_file(self, file_path):
        print(f"Load {file_path}")
        file_type = file_path.split('.')[-1]
        if file_type == 'csv':
            file = pd.read_csv(file_path, index_col=0)
        if file_type == 'pt':
            file = torch.load(file_path)
        return file

    def save_csv(self, df_file, file_name):
        full_name = os.path.join(self.path_inference_output, f"postprocess_data")
        os.makedirs(full_name, exist_ok=True)
        full_name = os.path.join(full_name, file_name)
        df_file.to_csv(full_name)

    def save_txt(self, str_txt, output_folder, file_name):
        full_name = os.path.join(output_folder, file_name)
        with open(full_name, "w") as text_file:
            text_file.write(str_txt)

    def insert_in_specific_col(self, all_frames_traject, frame_ind, curr_node, next_node):
        if curr_node in all_frames_traject[frame_ind, :]:
            flag = 0
            ind_place = np.argwhere(all_frames_traject[frame_ind, :] == curr_node)
            if frame_ind + 1 < all_frames_traject.shape[0]:
                all_frames_traject[frame_ind + 1, ind_place] = next_node
        else:
            flag = 1
            ind_place = np.argwhere(all_frames_traject[frame_ind, :] == -2)
            while ind_place.size == 0:
                new_col = -2 * np.ones((all_frames_traject.shape[0], 1), dtype=all_frames_traject.dtype)
                all_frames_traject = np.append(all_frames_traject, new_col, axis=1)
                ind_place = np.argwhere(all_frames_traject[frame_ind, :] == -2)
            ind_place = ind_place.min()
            all_frames_traject[frame_ind, ind_place] = curr_node
            if frame_ind + 1 < all_frames_traject.shape[0]:
                all_frames_traject[frame_ind + 1, ind_place] = next_node

        return flag, all_frames_traject

    def fill_first_frame(self, cell_starts):
        cols = ["child_id", "parent_id", "start_frame"]
        df_parent = pd.DataFrame(index=range(len(list(cell_starts))), columns=cols)
        df_parent.loc[:, ["start_frame", "parent_id"]] = 0
        df_parent.loc[:, "child_id"] = cell_starts
        return df_parent

    def find_parent_cell(self, frame_ind, all_frames_traject, df, num_starts, cell_starts):
        ind_place = np.argwhere(all_frames_traject[frame_ind, :] == -1)#轨迹结束位置
        finish_node_ids = all_frames_traject[frame_ind - 1, ind_place].squeeze(axis=1)#轨迹结束的位置与结束的nodes
        # print(f"frame_ind: {frame_ind}, cell_starts: {cell_starts}, cell_ends: {finish_node_ids}")

        df_parent = pd.DataFrame(index=range(len(cell_starts)), columns=self.cols)
        df_parent.loc[:, "start_frame"] = frame_ind#存在新细胞轨迹，保存开始帧数

        if finish_node_ids.shape[0] != 0:#存在轨迹结束
            if self.is_3d:
                finish_cell = df.loc[finish_node_ids, ["centroid_depth", "centroid_row", "centroid_col"]].values
            else:
                finish_cell = df.loc[finish_node_ids, ["centroid_row", "centroid_col"]].values#保存结尾id中心位置
            for ind, cell in enumerate(cell_starts):
                if self.is_3d:
                    curr_cell = df.loc[cell, ["centroid_depth", "centroid_row", "centroid_col"]].values
                else:
                    curr_cell = df.loc[cell, ["centroid_row", "centroid_col"]].values#保存新开始的细胞ID

                distance = ((finish_cell - curr_cell) ** 2).sum(axis=-1)
                nearest_cell = np.argmin(distance, axis=-1)
                parent_cell = int(finish_node_ids[nearest_cell])#最近的结束细胞为亲代
                df_parent.loc[ind, "child_id"] = cell
                df_parent.loc[ind, "parent_id"] = parent_cell
        else:
            df_parent.loc[:, "child_id"] = cell_starts
            df_parent.loc[:, "parent_id"] = 0

        return df_parent

    def clean_repetition(self, df):
        all_childs = df.child_id.values
        unique_vals, count_vals = np.unique(all_childs, return_counts=True)
        prob_vals = unique_vals[count_vals > 1]
        for prob_val in prob_vals:
            masking = df.child_id.values == prob_val
            all_apearence = df.loc[masking, :]
            start_frame = all_apearence.start_frame.min()
            end_frame = all_apearence.end_frame.max()
            df.loc[all_apearence.index[0], ["start_frame", "end_frame"]] = start_frame, end_frame
            df = df.drop(all_apearence.index[1:])

        return df.reset_index(drop=True)

    def set_all_info(self, df_parents_all, all_frames_traject):

        iterate_childs = df_parents_all.child_id.values
        frames_traject_same_label = all_frames_traject.copy()
        for ind, child_ind in enumerate(iterate_childs):
            # find the place where we store the child_ind in the trajectory matrix
            # validate that only one place exists
            coordinates_child = np.argwhere(all_frames_traject == child_ind)
            n_places = coordinates_child.shape[0]

            assert n_places == 1, f"Problem! find {n_places} places which the current child appears"

            coordinates_child = coordinates_child.squeeze()
            row, col = coordinates_child
            s_frame = df_parents_all.loc[ind, "start_frame"]
            assert row == s_frame, f"Problem! start frame {s_frame} is not equal to row {row}"

            # take the specific col from 'row' down
            curr_col = all_frames_traject[row:, col]
            last_ind = np.argwhere(curr_col == -1)
            if last_ind.size != 0:
                last_ind = last_ind[0].squeeze()
                curr_col = curr_col[:last_ind]
            e_frame = row + curr_col.shape[0] - 1

            df_parents_all.loc[ind, "end_frame"] = int(e_frame)
            # curr_id = curr_col[-1]#以最后一位结尾作为curr_id change
            curr_id = curr_col[0]
            # print(curr_id)
            df_parents_all.loc[ind, "child_id"] = curr_id
            frames_traject_same_label[row:e_frame + 1, col] = curr_id#以最后一位的node构建图像

        assert not(df_parents_all.isnull().values.any()), "Problem! dataframe contains NaN values"
        df_parents_all = self.clean_repetition(df_parents_all.astype(int))
        return df_parents_all.astype(int), frames_traject_same_label#如果结尾在同一个mask上，则导致两条细胞轨迹为同一mask值

    def df2str(self, df_track):
        """
        L B E P where
        L - a unique label of the track (label of markers, 16-bit positive value)
        B - a zero-based temporal index of the frame in which the track begins
        E - a zero-based temporal index of the frame in which the track ends
        P - label of the parent track (0 is used when no parent is defined)
        """
        str_track = ''
        for i in df_track.index:
            L = df_track.loc[i, "child_id"]
            B = df_track.loc[i, "start_frame"]
            E = df_track.loc[i, "end_frame"]
            P = df_track.loc[i, "parent_id"]
            str_track += f"{L} {B} {E} {P}\n"

        return str_track

    def merge_edges(self):
        in_output_pred, out_output_pred = self.match_edges()
        if self.merge_operation == 'OR' or self.merge_operation == 'AND':
            in_outputs_soft = torch.sigmoid(in_output_pred)
            in_outputs_hard = (in_outputs_soft > self.decision_threshold).int()

            out_outputs_soft = torch.sigmoid(out_output_pred)
            out_outputs_hard = (out_outputs_soft > self.decision_threshold).int()

            final_outputs_hard = np.bitwise_or(in_outputs_hard,out_outputs_hard) if self.merge_operation == 'OR'\
                else np.bitwise_and(in_outputs_hard,out_outputs_hard)

        if self.merge_operation == 'AVG':
            avg_outputs_soft = torch.sigmoid(in_output_pred) + torch.sigmoid(out_output_pred)
            avg_outputs_soft = avg_outputs_soft / 2.0
            final_outputs_hard = (avg_outputs_soft > self.decision_threshold).int()

        self.outputs_hard = final_outputs_hard
        return final_outputs_hard

    def megre_match_edges(self, edge_index, output_pred):

        assert torch.all(edge_index[:, ::2] == edge_index[[1, 0], 1::2]), \
            "The results don't match!"
        edge_index = edge_index[:, ::2]
        in_output_pred = output_pred[::2]
        out_output_pred = output_pred[1::2]

        if self.merge_operation == 'OR' or self.merge_operation == 'AND':
            in_outputs_soft = torch.sigmoid(in_output_pred)
            in_outputs_hard = (in_outputs_soft > self.decision_threshold).int()

            out_outputs_soft = torch.sigmoid(out_output_pred)
            out_outputs_hard = (out_outputs_soft > self.decision_threshold).int()

            final_outputs_hard = np.bitwise_or(in_outputs_hard, out_outputs_hard) if self.merge_operation == 'OR' \
                else np.bitwise_and(in_outputs_hard, out_outputs_hard)

        elif self.merge_operation == 'AVG':
            avg_outputs_soft = torch.sigmoid(in_output_pred) + torch.sigmoid(out_output_pred)
            avg_outputs_soft = avg_outputs_soft / 2.0
            final_outputs_hard = (avg_outputs_soft > self.decision_threshold).int()

        return final_outputs_hard, edge_index

    def find_connected_edges(self):
        edge_index, df, outputs = self.edge_index, self.df_preds, self.output_pred

        if not self.directed:
            final_outputs_hard, edge_index = self.megre_match_edges(edge_index.detach().clone(), outputs.detach().clone())
            self.outputs_hard = final_outputs_hard
            self.edge_index = edge_index
        else:
            outputs_soft = torch.sigmoid(outputs)
            self.outputs_hard = (outputs_soft > self.decision_threshold).int()
    
    def test(self):
        edge_index, df, outputs_hard = self.edge_index, self.df_preds, self.outputs_hard
        connected_indices = edge_index[:, outputs_hard.bool()]
        return edge_index, connected_indices

    def create_trajectory(self):
        edge_index, df, outputs_hard = self.edge_index, self.df_preds, self.outputs_hard
        self.flag_id0_terminate = False
        # extract values from arguments
        connected_indices = edge_index[:, outputs_hard.bool()]
        print(connected_indices)#change

        # find number of frames for iterations
        frame_nums = np.unique(df.frame_num)
        # find number of cells in each frame and build matrix [num_frames, max_cells]
        max_elements = [df.frame_num.isin([i]).sum() for i in frame_nums]
        all_frames_traject = np.zeros((frame_nums.shape[0], max(max_elements)))
        #创建轨迹图，横轴是帧数，纵轴是某一帧出现的最多细胞数 

        # intialize the matrix with -2 meaning empty cell, -1 means end of trajectory,
        # other value means the number of node in the graph
        all_frames_traject[:, :] = -2
        all_trajectory_dict = {}
        str_track = ''
        df_parents = []
        for frame_ind in frame_nums:
            mask_frame_ind = df.frame_num.isin([frame_ind])  # find the places containing frame_ind

            # filter the places with the specific frame_ind and take the corresponding indices
            nodes = df.loc[mask_frame_ind, :]
            nodes_indices = nodes.index.values
            #nodes_indices为当前帧的mask的唯一标识值

            next_frame_indices = np.array([])

            if frame_ind == 0:  # for the first frame, we should fill the first row with node indices
                all_frames_traject[frame_ind, :nodes_indices.shape[0]] = nodes_indices
                df_parents.append(self.fill_first_frame(nodes_indices))

            num_starts = 0
            cell_starts = []
            for i in nodes_indices:
                if i in connected_indices[0, :]:
                    ind_place = np.argwhere(connected_indices[0, :] == i)

                    if ind_place.shape[-1] > 1:
                        next_frame_ind = connected_indices[1, ind_place].numpy().squeeze()
                        if self.is_3d:
                            next_frame = df.loc[next_frame_ind, ["centroid_depth", "centroid_row", "centroid_col"]].values
                            curr_node = df.loc[i, ["centroid_depth", "centroid_row", "centroid_col"]].values
                        else:
                            next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values
                            curr_node = df.loc[i, ["centroid_row", "centroid_col"]].values

                        distance = ((next_frame - curr_node) ** 2).sum(axis=-1)
                        nearest_cell = np.argmin(distance, axis=-1)
                        # add to the array
                        next_node_ind = next_frame_ind[nearest_cell]

                    elif ind_place.shape[-1] == 1:  # one node in the next frame is connected to current node
                        next_node_ind = connected_indices[1, ind_place[0]]
                    else:  # no node in the next frame is connected to current node -
                        # in this case we end the trajectory
                        next_node_ind = -1

                else:
                    # we dont find the current node in the edge indices matrix - meaning we dont have a connection
                    # for the node - in this case we end the trajectory and the cell
                    if i == 0:
                        self.flag_id0_terminate = True
                    next_node_ind = -1

                next_frame_indices = np.append(next_frame_indices, next_node_ind)#保存连接结果
                # count the number of starting trajectories
                start, all_frames_traject = self.insert_in_specific_col(all_frames_traject, frame_ind, i, next_node_ind)
                num_starts += start

                if start == 1:  # append the id of the cell to the list 添加新的细胞nodes（表示新开始的序列）
                    cell_starts.append(i)

            if num_starts > 0:  #说明有新的细胞轨迹
                df_parents.append(self.find_parent_cell(frame_ind, all_frames_traject, df, num_starts, cell_starts))
            all_trajectory_dict[frame_ind] = {'from': nodes_indices, 'to': next_frame_indices}

        all_frames_traject = all_frames_traject.astype(int)

        # create csv contains all the relevant information for the res_track.txt
        df_parents_all = pd.concat(df_parents, axis=0).reset_index(drop=True)
        df_track_res, trajectory_same_label = self.set_all_info(df_parents_all, all_frames_traject)

        # convert csv to res_track.txt and res_track_real.txt
        str_track = self.df2str(df_track_res)

        # convert all_frames_traject to csv
        df_trajectory = pd.DataFrame(all_frames_traject)


        self.all_frames_traject = all_frames_traject
        self.trajectory_same_label = trajectory_same_label
        self.df_trajectory = df_trajectory
        self.df_track = df_track_res
        self.file_str = str_track

        return all_frames_traject, trajectory_same_label, \
               df_trajectory,  \
               str_track

    def get_pred(self, idx):
        pred = None
        if len(self.results):
            im_path = self.results[idx]
            pred = io.imread(im_path)
            if self.is_3d and len(pred.shape) != 3:
                pred = np.stack(imageio.mimread(im_path))
                assert len(pred.shape) == 3, f"Expected 3d dimiension! but {pred.shape}"
        return pred

    def create_save_dir(self):
        num_seq = self.dir_result.split('/')[-1][:2]
        save_tra_dir = osp.join(self.dir_result, f"../{num_seq}_RES")
        self.save_tra_dir =save_tra_dir
        os.makedirs(self.save_tra_dir, exist_ok=True)

    def save_new_pred(self, new_pred, idx):
        idx_str = "%03d" % idx
        file_name = f"mask{idx_str}.tif"
        full_dir = osp.join(self.save_tra_dir, file_name)
        io.imsave(full_dir, new_pred.astype(np.uint16))

    def check_ids_consistent(self, frame_ind, pred_ids, curr_ids):

        predID_not_in_currID = [x for x in pred_ids if x not in curr_ids]
        currID_not_in_predID = [x for x in curr_ids if x not in pred_ids]
        flag1 = len(predID_not_in_currID) == 1 and predID_not_in_currID[0] == 0
        flag2 = len(currID_not_in_predID) == 0
        if not flag2:
            # print('pred_ids', pred_ids)
            # print('curr_ids:', curr_ids)
            print('currID_not_in_predID', currID_not_in_predID)
        if not flag1:
            str_print = f"Frame {frame_ind}: Find segmented cell {predID_not_in_currID} without assigned labels"
            warnings.warn(str_print)
            #change
        # assert flag2, f"Frame {frame_ind}: Find assigned labels {currID_not_in_predID} " \
        #               f"which are not appears in the final saved results"
        flag1 = 0
        flag2 = 0

        return flag1, predID_not_in_currID

    def fix_inconsistent(self, pred_prob_ids, pred):
        for id in pred_prob_ids:
            if id == 0:
                continue
            pred[pred == id] = 0
        return pred

    def fill_mask_labels(self, debug=False):
        self.create_save_dir()
        all_frames_traject, trajectory_same_label = self.all_frames_traject, self.trajectory_same_label
        df = self.df_preds
        n_rows, n_cols = all_frames_traject.shape
        self.flag_id0_terminate = True
        #change

        count_diff_vals = 0
        for idx in range(n_rows):
            pred = self.get_pred(idx)#得到相应帧数的图片
            pred_copy = pred.copy()
            curr_row = all_frames_traject[idx, :]#得到当前行
            mask_id = np.bitwise_and(curr_row != -1, curr_row != -2)#选取不为空的元素位置
            graph_ids = curr_row[mask_id]#得到mask值（all_frames_traject）
            graph_true_ids = trajectory_same_label[idx, mask_id]#same_id里面得到（即轨迹的结束ID）
            mask_where = np.ones_like(pred)#设置0和1的图像（保存位置
            frame_ids = []
            for id, true_id in zip(graph_ids, graph_true_ids):
                #id mask 的唯一标识
                #true_id 要修改的值
                flag_id0 = true_id == 0
                if flag_id0:    # edge case when the cell with id=0 terminate after one frame
                    if self.flag_id0_terminate:
                        new_id = trajectory_same_label.max() + 1
                        self.df_track.child_id[self.df_track.child_id == 0] = new_id#修改trueID=0为newID
                        self.file_str = self.df2str(self.df_track)
                    else:
                        assert False, "Problem!"
                if self.is_3d:
                    cell_center = df.loc[id, ["centroid_depth", "centroid_row", "centroid_col"]].values.astype(int)
                    depth_center, row_center, col_center = cell_center[0], cell_center[1], cell_center[2]
                    if self.center_coord:
                        n_depth_img, n_row_img, n_col_img = pred.shape
                        depth_center += n_depth_img // 2
                        row_center += n_row_img // 2
                        col_center += n_col_img // 2

                    val = pred[depth_center, row_center, col_center]
                    if 'seg_label' in df.columns:
                        v_old = val
                        val = df.loc[id, "seg_label"]
                        count_diff_vals += 1 if v_old != val else 0

                    if val == 0:
                        if np.any(pred[depth_center-3:depth_center+3, row_center-3:row_center+3, col_center-3:col_center+3] != 0):
                            area = pred[depth_center-3:depth_center+3, row_center-3:row_center+3, col_center-3:col_center+3]
                            unique_labels, counts = np.unique(area, return_counts=True)
                            mask = unique_labels != 0
                            unique_labels = unique_labels[mask]
                            counts = counts[mask]
                            val = unique_labels[np.argmax(counts)]
                        else:
                            print("Problem! The provided center coordinates value is zero, should be labeled with other value")
                            print(df.loc[id, ["seg_label", "frame_num",  "centroid_depth", "centroid_row", "centroid_col", "min_depth_bb",
                                              "min_row_bb", "min_col_bb", "max_depth_bb", "max_row_bb", "max_col_bb"]].astype(int))
                            print()
                            continue
                else:
                    cell_center = df.loc[id, ["centroid_row", "centroid_col"]].values.astype(int)#通过df找到中心位置
                    row_center, col_center = cell_center[0], cell_center[1]
                    if self.center_coord:
                        n_row_img, n_col_img = pred.shape
                        row_center += n_row_img // 2
                        col_center += n_col_img // 2

                    val = pred[row_center, col_center]
                    if 'seg_label' in df.columns:
                        v_old = val
                        val = df.loc[id, "seg_label"]
                        count_diff_vals += 1 if v_old != val else 0

                    if val == 0:
                        print('val = 0')
                        if np.any(pred[row_center-3:row_center+3, col_center-3:col_center+3] != 0):
                            area = pred[row_center-3:row_center+3, col_center-3:col_center+3]
                            unique_labels, counts = np.unique(area, return_counts=True)
                            mask = unique_labels != 0
                            unique_labels = unique_labels[mask]
                            counts = counts[mask]
                            val = unique_labels[np.argmax(counts)]
                        else:
                            print(
                                "Problem! The provided center coordinates value is zero, should be labeled with other value")
                            print(df.loc[id, ["seg_label", "frame_num", "centroid_row", "centroid_col",
                                              "min_row_bb", "min_col_bb", "max_row_bb", "max_col_bb"]].astype(int))
                            print()
                            continue
                        #这段代码检查 val 是否等于 0，如果是，则说明当前像素点的预测值为 0。在这种情况下，代码尝试找到周围 3x3 区域内非零的像素值，并选择其中最常出现的像素值作为新的 val。

                assert val != 0, "Problem! The provided center coordinates value is zero, " \
                                 "should be labeled with other value"
                if flag_id0:
                    true_id = new_id
                mask_val = (pred_copy == val).copy()
                mask_curr = np.logical_and(mask_val, mask_where)
                pred_copy[mask_curr] = true_id
                # if not np.any(mask_curr):
                #     print('empty1')
                # if idx == 4 :
                #     plt.subplot(2,2,3)
                #     plt.imshow(pred_copy)
                #     if true_id == 113:
                #         if not np.any(mask_curr):
                #             print('empty2')
                #         plt.subplot(2,2,1)
                #         plt.imshow(pred_copy)
                #         plt.subplot(2,2,2)
                #         plt.imshow(mask_curr)
                #         # plt.show()

                mask_where = np.logical_and(np.logical_not(mask_val), mask_where)

                frame_ids.append(true_id)

            isOK, predID_not_in_currID = self.check_ids_consistent(idx, np.unique(pred_copy), frame_ids)
            if not debug:
                if not isOK:
                    pred_copy = self.fix_inconsistent(predID_not_in_currID, pred_copy)
                self.save_new_pred(pred_copy, idx)
        print(f"Number of different vals: {count_diff_vals}")
        self.save_txt(self.file_str, self.save_tra_dir, 'res_track.txt')


modality = '2D'
path_inference_output = '/data/sunrui/celldata/r03c06f04ch1/01_RES_inference'
path_Seg_result = '/data/sunrui/celldata/r03c06f04ch1/01_GT/SEG/'
is_3d = '3d' in modality.lower()
directed = True
merge_operation = 'AND'
pp = Postprocess(is_3d=is_3d,
                    type_masks='tif', merge_operation=merge_operation,
                    decision_threshold=0.5,
                    path_inference_output=path_inference_output, center_coord=False,
                    directed=directed,
                    path_seg_result=path_Seg_result)

all_frames_traject, trajectory_same_label, df_trajectory, str_track = pp.create_trajectory()
pp.fill_mask_labels(debug=False)

/home/sunrui/anaconda3/envs/cell-tracking-challenge/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/pytorch_geometric_data.pt
Load /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/all_data_df.csv
Load /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/raw_output.pt
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])
currID_not_in_predID [113]
currID_not_in_predID [41, 113]
currID_not_in_predID [41, 113]
currID_not_in_predID [41, 113]
currID_not_in_predID [41, 113]
currID_not_in_predID [41, 113]
currID_not_in_predID [41, 43, 113]
currID_not_in_predID [43, 113]


KeyboardInterrupt: 

In [30]:
modality = '2D'
path_inference_output = '/data/sunrui/celldata/r03c06f04ch1/01_RES_inference'
path_Seg_result = '/data/sunrui/celldata/r03c06f04ch1/01_GT/SEG/'
is_3d = '3d' in modality.lower()
directed = True
merge_operation = 'AND'
pp = Postprocess(is_3d=is_3d,
                    type_masks='tif', merge_operation=merge_operation,
                    decision_threshold=0.5,
                    path_inference_output=path_inference_output, center_coord=False,
                    directed=directed,
                    path_seg_result=path_Seg_result)

all_frames_traject, trajectory_same_label, df_trajectory, str_track = pp.create_trajectory()
pp.fill_mask_labels(debug=False)

Load /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/pytorch_geometric_data.pt
Load /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/all_data_df.csv
Load /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/raw_output.pt
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])


KeyboardInterrupt: 

In [14]:
x1,x2 = pp.test()
print(x1)
print(x2)

tensor([[    0,     0,     0,  ..., 16146, 16146, 16146],
        [   53,    55,    74,  ..., 16154, 16164, 16171]])
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])


In [15]:
print(all_frames_traject)

[[    0     1     2 ...    -2    -2    -2]
 [   53    55    54 ...    -2    -2    -2]
 [  110   109    -1 ...    -2    -2    -2]
 ...
 [16088 16082 16101 ...    -2    -2    -2]
 [16124 16118 16139 ...    -2    -2    -2]
 [16158 16151 16154 ...    -2    -2    -2]]


In [16]:
trajectory_same_label
df_same_tra = pd.DataFrame(trajectory_same_label)

In [17]:
df_same_tra.to_csv("/data/sunrui/celldata/r03c06f04ch1/df_same_tra.csv")

In [ ]:
df_trajectory

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,0,1,2,3,4,5,6,7,8,9,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
1,53,55,54,56,57,59,60,58,62,64,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
2,110,109,-1,111,159,114,112,119,115,121,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
3,167,166,-2,168,213,170,171,180,169,184,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
4,224,223,270,222,262,227,229,228,226,241,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,16028,16023,16026,16020,16028,16028,16028,16039,-1,-1,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
325,16054,16047,16052,16059,16054,16054,16054,16069,16049,16051,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
326,16088,16082,16101,16106,16088,16088,16088,16103,16086,16092,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
327,16124,16118,16139,16138,16124,16124,16124,16135,16125,-1,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2


In [ ]:

df_trajectory.to_csv("/data/sunrui/celldata/r03c06f04ch1/df_tra.csv")

In [ ]:
str_track

'1912 0 36 0\n8318 0 151 0\n8354 0 151 0\n12449 0 227 0\n8327 0 151 0\n15765 0 315 0\n12400 0 225 0\n8321 0 151 0\n8324 0 151 0\n8362 0 151 0\n8328 0 151 0\n8350 0 151 0\n12432 0 226 0\n3711 0 69 0\n8346 0 151 0\n8333 0 151 0\n8332 0 151 0\n2978 0 56 0\n8330 0 151 0\n15319 0 302 0\n8326 0 151 0\n8336 0 151 0\n8343 0 151 0\n8319 0 151 0\n16149 0 328 0\n8344 0 151 0\n8329 0 151 0\n8369 0 151 0\n2016 0 37 0\n8338 0 151 0\n8339 0 151 0\n8356 0 151 0\n597 0 10 0\n607 0 10 0\n441 1 7 0\n609 1 10 0\n372 4 6 0\n611 8 10 441\n553 9 9 0\n769 11 13 611\n8341 12 151 0\n8371 12 151 0\n934 15 16 0\n1403 25 25 0\n8352 28 151 0\n8357 28 151 0\n1758 32 32 0\n2535 47 47 0\n2589 49 49 0\n8359 50 151 2589\n2975 52 55 0\n8358 62 151 0\n3691 63 68 0\n3412 63 63 0\n3650 67 67 0\n4050 71 74 0\n8351 72 151 0\n8337 74 151 0\n4161 76 76 0\n8355 78 151 0\n4267 78 78 0\n4323 79 79 4267\n4432 80 81 4323\n4434 81 82 0\n4544 84 84 0\n8379 84 152 0\n8347 84 151 0\n4711 86 86 0\n4829 87 88 4711\n4827 88 88 0\n6274 95 1

In [ ]:
# for frame_ind in frame_nums:
#     mask_frame_ind = df.frame_num.isin([frame_ind])  # 找到包含特定帧编号的位置

#     # 过滤出特定帧编号的位置，并获取相应的节点数据
#     nodes = df.loc[mask_frame_ind, :]
#     nodes_indices = nodes.index.values

#     next_frame_indices = np.array([])

#     if frame_ind == 0:  # 对于第一帧，需要用节点索引填充第一行
#         all_frames_traject[frame_ind, :nodes_indices.shape[0]] = nodes_indices
#         df_parents.append(self.fill_first_frame(nodes_indices))

#     num_starts = 0
#     cell_starts = []
    
#     # 遍历当前帧的每个节点
#     for i in nodes_indices:
#         if i in connected_indices[0, :]:  # 如果当前节点连接到下一帧的节点
#             ind_place = np.argwhere(connected_indices[0, :] == i)

#             if ind_place.shape[-1] > 1:  # 当前节点连接多个下一帧的节点
#                 next_frame_ind = connected_indices[1, ind_place].numpy().squeeze()
#                 if self.is_3d:
#                     next_frame = df.loc[next_frame_ind, ["centroid_depth", "centroid_row", "centroid_col"]].values
#                     curr_node = df.loc[i, ["centroid_depth", "centroid_row", "centroid_col"]].values
#                 else:
#                     next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values
#                     curr_node = df.loc[i, ["centroid_row", "centroid_col"]].values

#                 distance = ((next_frame - curr_node) ** 2).sum(axis=-1)
#                 nearest_cell = np.argmin(distance, axis=-1)
#                 next_node_ind = next_frame_ind[nearest_cell]

#             elif ind_place.shape[-1] == 1:  # 当前节点连接一个下一帧的节点
#                 next_node_ind = connected_indices[1, ind_place[0]]
#             else:  # 没有节点连接到当前节点，结束轨迹
#                 next_node_ind = -1

#         else:
#             # 当前节点在边索引矩阵中没有连接节点 - 表示没有连接
#             # 对于节点 i == 0 的情况，结束轨迹
#             if i == 0:
#                 self.flag_id0_terminate = True
#             next_node_ind = -1

#         next_frame_indices = np.append(next_frame_indices, next_node_ind)
        
#         # 计算开始轨迹的数量并更新轨迹矩阵
#         start, all_frames_traject = self.insert_in_specific_col(all_frames_traject, frame_ind, i, next_node_ind)
#         num_starts += start

#         if start == 1:  # 将细胞的 ID 添加到列表中
#             cell_starts.append(i)

#     # 如果存在开始的轨迹，找到并添加父细胞的信息
#     if num_starts > 0:
#         df_parents.append(self.find_parent_cell(frame_ind, all_frames_traject, df, num_starts, cell_starts))
    
#     # 为当前帧添加轨迹信息到字典中
#     all_trajectory_dict[frame_ind] = {'from': nodes_indices, 'to': next_frame_indices}


In [ ]:
if __name__== "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('-modality', type=str, required=True, help='2D/3D modality')
    parser.add_argument('-iseg', type=str, required=True, help='segmentation output directory')
    parser.add_argument('-oi', type=str, required=True, help='inference output directory')

    args = parser.parse_args()

    modality = args.modality
    assert modality == '2D' or modality == '3D'

    path_inference_output = args.oi
    path_Seg_result = args.iseg

    is_3d = '3d' in modality.lower()
    directed = True
    merge_operation = 'AND'

    pp = Postprocess(is_3d=is_3d,
                     type_masks='tif', merge_operation=merge_operation,
                     decision_threshold=0.5,
                     path_inference_output=path_inference_output, center_coord=False,
                     directed=directed,
                     path_seg_result=path_Seg_result)
    all_frames_traject, trajectory_same_label, df_trajectory, str_track = pp.create_trajectory()
    pp.fill_mask_labels(debug=False)

usage: ipykernel_launcher.py [-h] -modality MODALITY -iseg ISEG -oi OI
ipykernel_launcher.py: error: the following arguments are required: -modality, -iseg, -oi


SystemExit: 2